
> **Distribution A**
> Approved for Public Release, Distribution Unlimited

# Workflow for reproducing *Interpretable models for extrapolation in scientific machine learning*

General procedure:
* Import the config file which contains dataset information
* Import all datasets from CSV files as Pandas DataFrames
* Featurize the Matminer datasets using Magpie features
* Clean all datasets
* Export all the clean featurized datasets to a single JSON file
* Partition the datasets into train-test-validation sets using LOCO CV (for extrapolation) and random CV (for interpolation)
* Perform combinatorial feature engineering on the datasets
    * for each dataset:
        * for each CV strategy:
            * for each train-test split:
                * perform feature engineering
                * rank engineered features
                * save top 10 engineered features
* Train and test ML models and linear regressions using original features and engineered features
* Analyze results of the model comparison

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
from scipy import stats
from collections import Counter
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

import utils

FONTSIZE = 10
LINEWIDTH = 0.5
TICKWIDTH = 0.5
plt.rcParams.update(
    {
        "xtick.labelsize": FONTSIZE,
        "ytick.labelsize": FONTSIZE,
        "axes.linewidth": LINEWIDTH,
        "xtick.minor.width": TICKWIDTH,
        "xtick.major.width": TICKWIDTH,
        "ytick.minor.width": TICKWIDTH,
        "ytick.major.width": TICKWIDTH,
        "font.family": "Arial",
        "figure.facecolor": "w",
        "figure.dpi": 600,
    }
)

# Create and view datasets

In [ ]:
utils.create_datasets()

# Get cross-validation splits

In [ ]:
# import datasets and create CV splits
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
splits = utils.get_cv_splits(ds, klims=(3, 10))

# use this to view the size of each split
#utils.view_split_sizes(splits)

utils.write_jsonzip(splits, os.path.join("data", "splits.json.gz"))

# Visualize LOCO splits

In [ ]:
utils.view_all_splits("LOCO")

# Visualize Random splits

In [ ]:
utils.view_all_splits("Random")

# Engineer new features

In [ ]:
# import the data
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
splits = utils.read_jsonzip(os.path.join("data", "splits.json.gz"))
split_types = ["LOCO", "Random"]

# create new engineered features
N_COMBOS = 500000  # number of combos to calculate
N_LIMS_PER_COMBO = (2, 5)  # min and max number of columns per combo
N_SAVE = 10  # number of top discovered features to save for each split

fs = {dsn: {} for dsn in ds}
starttime = time.time()

# loop over each dataset
for di, dsn in enumerate(list(ds)):

    # get dataset and r^2 between the best original input feature and the target
    print(f"{di+1}: {dsn}")
    df = pd.DataFrame(json.loads(ds[dsn]["df"]))
    t = ds[dsn]["target"]
    target_vals = df[t].values
    prev_rtot = (
        df.corrwith(df[t], method=utils.get_r2).sort_values(ascending=False).iloc[1]
    )

    # create additional input columns based on existing columns
    dfa = utils.add_additional_cols(df, ignore=[t])
    n_inputs_for_mixing = dfa.shape[1] - 1
    fs[dsn]["n_inputs_for_mixing"] = n_inputs_for_mixing
    print(f"{dsn}: {n_inputs_for_mixing} inputs")

    # create random combinations of input columns
    col_combos = utils.random_combos(
        inputs=list(set(utils.get_num_cols(dfa, ignore=[t]))),
        n_combos=N_COMBOS,
        lims=N_LIMS_PER_COMBO,
    )

    # loop over each split to find the best new features for that split
    for split_type in split_types:
        fs[dsn][split_type] = []

        print(f"  {split_type}")
        ss = splits[dsn][split_type]

        for si, s in enumerate(ss):

            # get indices of this validation set
            valid_idx = [df.index.tolist().index(i) for i in s["validation"]]
            target_valid = target_vals[valid_idx]

            correlation_dicts = []

            # loop over each new column combo and get validation set at this split
            for cc in col_combos:

                # get values of then new feature and target at this validation set
                data = utils.vals_from_combos(dfa, cc)
                if data is None:
                    continue

                data_valid = data[valid_idx]
                if not utils.unique_enough(data_valid):
                    continue

                # get r^2 correlations between the target and each engineered feature
                correlation = utils.get_r2(data_valid, target_valid)
                if np.isfinite(correlation):
                    correlation_dicts.append({"feat": cc, "rtot": correlation})

            # sort by the most highly correlated features
            correlation_dicts = sorted(
                correlation_dicts, key=lambda d: d["rtot"], reverse=True
            )[:N_SAVE]

            # save results to dataframe
            for cd in correlation_dicts:
                cd["val"] = utils.vals_from_combos(dfa, cd["feat"])
                cd["feat"] = utils.refactor_feat(cd["feat"])
            corrs = pd.DataFrame(correlation_dicts)
            fs[dsn][split_type].append(corrs.to_json(default_handler=str))

            # determine whether new features beat the old features
            new_rtot = corrs["rtot"].iloc[0]
            if new_rtot < prev_rtot:
                print(f"\tsplit {si}: no improvement")
            else:
                print(
                    f"\tsplit {si}: r^2 {round(prev_rtot, 2)} -> {round(new_rtot, 2)}"
                )

print(f"\nFeature discovery runtime: {round((time.time() - starttime)/60, 2)} min")
# save features to file
utils.write_jsonzip(fs, os.path.join("data", "features.json.gz"))

# Plot target vs best feature for all datasets and LOCO splits

In [ ]:
utils.plot_target_vs_best_feature()

# Plot feature correlations with targets

In [ ]:
def plot_feature_correlations_w_targets():
    """Plot Pearson and Spearman correlations between
    all input features and the target feature"""
    fig, ax = plt.subplots(nrows=3, ncols=3)
    ax = np.ravel(ax)
    ii = 0
    CV_COLORS = {
        "Original": "black",
        "LOCO": "red",
        "Random": "blue",
    }
    ALPHA = 0.2

    ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
    fs = utils.read_jsonzip(os.path.join("data", "features.json.gz"))

    for dsn in ds:
        print(f"\n{dsn}")
        t = ds[dsn]["target"]
        df = pd.DataFrame(json.loads(ds[dsn]["df"]))

        for split_type in ["Original", "LOCO", "Random"]:
            r2s, sr2s = [], []

            if split_type == "Original":
                for c in utils.get_num_cols(df, ignore=[t]):
                    x, y = df[c].values, df[t].values
                    r2s.append(np.square(stats.pearsonr(x, y)[0]))
                    sr2s.append(np.square(stats.spearmanr(x, y)[0]))
                best_idx = np.argmax(r2s)
                highlight = (sr2s[best_idx], r2s[best_idx])

            else:
                # get pearson and spearman r^2 values across all new features
                new_feat_df = pd.concat(
                    [pd.DataFrame(json.loads(i)) for i in fs[dsn][split_type]]
                )
                for i, row in new_feat_df.iterrows():
                    x, y = row["val"], df[t].values
                    r2s.append(np.square(stats.pearsonr(x, y)[0]))
                    sr2s.append(np.square(stats.spearmanr(x, y)[0]))
                best_idx = np.argmax(r2s)
                highlight = (np.median(sr2s), np.median(r2s))

            # plot highlighted location
            ax[ii].scatter(
                highlight[0],
                highlight[1],
                s=150,
                c=CV_COLORS[split_type],
                marker="+",
                lw=1,
                zorder=99,
            )

            # plot all features
            ax[ii].scatter(
                sr2s,
                r2s,
                lw=0.25,
                s=6,
                edgecolors="w",
                alpha=ALPHA,
                c=CV_COLORS[split_type],
            )
            print(
                f"{split_type} best r2: {round(r2s[best_idx], 3)}, best sr2: {round(sr2s[best_idx], 3)}"
            )

            # add confidence ellipse
            utils.confidence_ellipse(
                sr2s,
                r2s,
                ax[ii],
                n_std=1,
                facecolor="none",
                lw=0.5,
                edgecolor=CV_COLORS[split_type],
                linestyle="solid",
            )

        ax[ii].tick_params(labelsize=FONTSIZE)
        # ax[ii].set_xticks([0, 0.5, 1], [0, 0.5, 1])
        # ax[ii].set_yticks([0, 0.5, 1], [0, 0.5, 1])
        ax[ii].set_xlim([0, 1])
        ax[ii].set_ylim([0, 1])

        if ii in [0, 3, 6]:
            ax[ii].set_ylabel("r$^2$", fontsize=FONTSIZE)

        else:
            ax[ii].set_ylabel("")
            ax[ii].set_yticklabels([])
        if ii in [6, 7, 8]:
            ax[ii].set_xlabel("ρ$^2$", fontsize=FONTSIZE)
        else:
            ax[ii].set_xlabel("")
            ax[ii].set_xticklabels([])

        ax[ii].text(
            0.03,
            0.96,
            t,
            fontsize=FONTSIZE,
            ha="left",
            va="top",
            transform=ax[ii].transAxes,
        )

        ax[ii].axvline(
            x=0.5, color="gray", lw=1, linestyle="dotted", alpha=0.3, zorder=0
        )
        ax[ii].axhline(
            y=0.5, color="gray", lw=1, linestyle="dotted", alpha=0.3, zorder=0
        )
        ii += 1

    ax[1].legend(
        ncol=3,
        bbox_to_anchor=(0.5, 1.05),
        loc="lower center",
        # fontsize=16,
        handles=[
            mpatches.Patch(
                color=CV_COLORS[k],
                alpha=1,
                label=k,
            )
            for k in CV_COLORS
        ],
    )
    plt.subplots_adjust(wspace=-1.2, hspace=0.4)
    plt.tight_layout()
    plt.gcf().savefig(os.path.join(utils.FIG_BASEPATH, "FeatureCorrelations.png"))
    plt.show()


plot_feature_correlations_w_targets()

# Get frequency of variables in top-performing features across all CV splits

In [ ]:
def get_base_var(x: str) -> str:
    """get base variable from engineered variable"""
    return x.split(" ")[-1].split("**")[0].replace("ln ", "")


# import the data
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
splits = utils.read_jsonzip(os.path.join("data", "splits.json.gz"))
fs = utils.read_jsonzip(os.path.join("data", "features.json.gz"))
split_types = ["LOCO", "Random"]
CV_COLORS = {"LOCO": "tomato", "Random": "dodgerblue"}

num_input_col_dict = {dsn: v["n_inputs_for_mixing"] for dsn, v in fs.items()}

# create figure for target value histograms
fig, ax = plt.subplots(nrows=3, ncols=3)
ax = np.ravel(ax)
ii = 0

n = 5  # number of features to plot in each panel
colors = cm.Set2(np.linspace(0, 1, n))[::-1]

# loop over each dataset
for di, dsn in enumerate(list(ds)):
    df = pd.DataFrame(json.loads(ds[dsn]["df"]))
    t = ds[dsn]["target"]

    # total number of input features in this dataset
    n_inputs = len(utils.get_num_cols(df)) - 1

    for st in split_types:

        # get all the feature combos saved for this dataset and split
        feat_list = []

        for si in range(len(fs[dsn][st])):
            featdf = pd.DataFrame(json.loads(fs[dsn][st][si]))

            feat_list += [ff for fl in list(featdf["feat"]) for ff in fl]

        # get ranking of frequencies at which each variable shows up in engineered features
        var_freqs = pd.DataFrame(
            data=Counter(feat_list).most_common(), columns=["feat", "freq"]
        )

        # expectation value is the total number of splits a feature should show up in on average.
        # use this number as the denominator to calculate the fraction of splits.
        # n_splits per strategy * n_top_features per split *
        # 3.5 (mean # of cols per engineered feat) /
        # number of possible input cols
        expectation_value = 10 * 10 * 3.5 / num_input_col_dict[dsn]
        x, y = list(var_freqs["feat"])[:n][::-1], var_freqs["freq"][:n][::-1].values

        # print(dsn, st, y)
        # print(expectation_value)
        if st == "LOCO":
            loco_vals = y / expectation_value
            bars_loco = ax[ii].barh(
                np.arange(len(x)),
                y / expectation_value,
                height=0.5,
                lw=0,
                alpha=0.75,
                color=CV_COLORS["LOCO"],
            )
        else:
            bars_random = ax[ii].barh(
                np.arange(len(x)),
                y / expectation_value,
                height=0.5,
                lw=0,
                alpha=0.75,
                left=loco_vals,
                color=CV_COLORS["Random"],
            )

    ax[ii].set_yticks(np.arange(len(x)))
    ax[ii].set_yticklabels(utils.format_labels(x))
    ax[ii].set_ylim([-0.4, len(x) - 0.6])
    ax[ii].set_xlim([0, 100])
    ax[ii].set_xticks([0, 25, 50, 75, 100])

    for x0 in [25, 50, 75]:
        ax[ii].axvline(x=x0, linestyle="solid", lw=0.1, c="gray")


    base_vars = [get_base_var(x0) for x0 in x]
    for bi in range(len(bars_loco)):
        if base_vars.count(get_base_var(x[bi])) > 1:

            ax[ii].get_yticklabels()[bi].set_color("purple")

    if ii > 5:
        ax[ii].set_xticklabels([0, 25, 50, 75, 100])
    else:
        ax[ii].set_xticklabels([])

    ax[ii].text(
        0.5,
        1.03,
        t,
        horizontalalignment="center",
        fontsize=FONTSIZE,
        transform=ax[ii].transAxes,
    )

    ii += 1


ax[1].legend(
    ncol=2,
    bbox_to_anchor=(0.5, 1.15),
    loc="lower center",
    # fontsize=16,
    handles=[
        mpatches.Patch(
            color=CV_COLORS[k],
            alpha=1,
            label=k,
        )
        for k in CV_COLORS
    ],
)

ax[7].set_xlabel("Frequency / expectation value", fontsize=FONTSIZE)
fig.set_size_inches(9, 5)
plt.subplots_adjust(wspace=1.7, hspace=0.5)
# plt.xlabel("Frequency")
plt.tight_layout()
plt.gcf().savefig(
    os.path.join(utils.FIG_BASEPATH, "FeatureVariableProminence.png"),
    bbox_inches="tight",
)
plt.show()

# Display best features based on their correlation to full dataset

In [ ]:
# read datasets
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
fs = utils.read_jsonzip(os.path.join("data", "features.json.gz"))
split_types = ["LOCO", "Random"]

for dsn in fs:
    t = ds[dsn]["target"]
    print(t)
    
    # get full dataset with augmented features
    df = pd.DataFrame(json.loads(ds[dsn]["df"]))
    dfa = utils.add_additional_cols(df, ignore=[t])
    target_vals = df[t]

    r2max = 0
    best_feat = None
    
    # iterate over all the engineered features in this dataset
    # iterate over split type
    for st in split_types:
        # iterate over each split
        for si in range(len(fs[dsn][st])):
            # iterate over the 10 engineered feastures saved for this split
            feats = [v for _, v in json.loads(fs[dsn][st][si])["feat"].items()]
            for feat in feats:
                
                # get correlation of this feature with the full dataset
                feature_vals = utils.vals_from_combos(dfa, feat)
                r2 = utils.get_r2(target_vals, feature_vals)
                
                if r2 > r2max:
                    r2max = r2
                    best_feat = feat
                    
    
    print(f"  {round(r2max, 2)} r2 to full dataset")
    for f0 in best_feat:
        print(f"    {f0}")

# Train and test ML models

In [ ]:
@ignore_warnings(category=ConvergenceWarning)
def train_model(
    df: pd.DataFrame,
    input_cols: list,
    target_col: str,
    splits: list,
    model_type: str,
    single_feat_vals: list,
    plot_pva: bool = False,
    scale: bool = True,
) -> dict:
    """
    Train an ML model using a dtafrme, target column,
    and dict of train-test splits.

    Inputs:
    df: dataframe containing the data
    input_cols: column names which should be useed as model input feautures
    target_col: column name to use as model output
    model_type: type of model. Choose one of ["NN", "RF", "linear"]
    splits: a list of train-test splits to use for model cross-validation.

    Output:
    results: dataframe summarizing the results of model testing
    model: trained model
    """

    print(
        f"    inputs: {len(input_cols)}, model: {model_type}, splits: {len(splits)}, samples: {len(df)}"
    )

    if model_type == "NN":
        model = MLPRegressor(random_state=1)
    elif model_type == "RF":
        # model = RandomForestRegressorCov(random_state=1)
        model = RandomForestRegressor(random_state=0)
    elif model_type == "linear":
        model = LinearRegression()
    else:
        raise ValueError(
            "model_type is not specified correctly. Try 'linear', 'NN', or 'RF'"
        )

    starttime = time.time()
    results = {k: [] for k in ["split", "actual", "predicted"]}
    # loop over each holdout set for cross-validation
    for si, s in enumerate(splits):

        # add best feat values for this split
        # found from feature engineering
        df["single"] = single_feat_vals[si]

        # scale the input data
        if scale:
            scaler = RobustScaler()
            dfs = pd.DataFrame(
                index=df.index,
                data=scaler.fit_transform(df.values),
                columns=df.columns,
            )

        # get indices of training and testing rows
        test_idx = splits[s]["test"]
        train_idx = splits[s]["train"]
        # get training and testing inputs and outputs
        train_in = dfs.loc[train_idx][input_cols].values
        train_out = dfs.loc[train_idx][target_col].values
        test_in = dfs.loc[test_idx][input_cols].values
        test_out = dfs.loc[test_idx][target_col].values

        # fit model and make predictions on test samples
        model.fit(train_in, train_out)

        if model_type == "RF":
            # for custom random forest covariance models
            # importances = list(model.feature_importances_)
            # pred, unc, _ = model.predict(test_in)
            pred = model.predict(test_in)
        elif model_type == "NN":
            # for neural nets
            # importances = [0]*len(input_cols)
            pred = model.predict(test_in)
        elif model_type == "linear":
            # for linear models
            # importances = model.coef_
            pred = model.predict(test_in)

        # save results
        results["split"] += [s] * len(test_out)
        results["actual"] += list(test_out)
        results["predicted"] += list(pred)

    return results


def get_model_inputs(input_type: str, df: pd.DataFrame, target: str) -> list:
    """Get the type of inputs to use for ML"""
    if input_type == "single":
        cols = ["single"]
    elif input_type == "original":
        cols = [c for c in df if all(["**" not in c, "ln " not in c, c != "single"])]
    elif input_type == "all":
        cols = list(df)
    else:
        raise ValueError("Input type is incorrect.")
    return [c for c in cols if c != target]


# read datasets
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
# read feature sets
fs = utils.read_jsonzip(os.path.join("data", "features.json.gz"))
# read splits
splits = utils.read_jsonzip(os.path.join("data", "splits.json.gz"))
split_types = ["LOCO", "Random"]
res = []
start_all = time.time()

# loop over each dataset
for di, dsn in enumerate(list(ds)):
    starttime = time.time()
    print(f"\n{di+1}/{len(ds)}: {dsn}")

    # get the data
    df = pd.DataFrame(json.loads(ds[dsn]["df"]))
    t = ds[dsn]["target"]

    # loop over each model type
    for model_type in ["NN", "RF", "linear"]:

        # loop over each split
        for split_type in split_types:
            print(f"  {split_type}")
            ss = splits[dsn][split_type]

            # this is a list of lists of the discovered feature values for
            # this dataset and CV strategy. each list of values corrresponds
            # to the best feature discovered at a single CV split
            disc_vals_list = [
                pd.DataFrame(json.loads(fs[dsn][split_type][i])).iloc[0]["val"]
                for i in range(10)
            ]

            # loop over each featurization strategy
            for input_type in ["single", "all", "original"]:

                # get model input columns
                input_cols = get_model_inputs(
                    input_type=input_type,
                    df=df,
                    target=t,
                )

                # print(input_cols)
                # train and test model
                results0 = train_model(
                    df=df,
                    input_cols=input_cols,
                    target_col=t,
                    splits={f"split-{si}": ss[si] for si in range(len(ss))},
                    model_type=model_type,
                    single_feat_vals=disc_vals_list,
                    scale=True,
                )
                # append results
                res.append(
                    {
                        "dataset": dsn,
                        "model": model_type,
                        "split_type": split_type,
                        "input_type": input_type,
                        "results": results0,
                    }
                )
    print(f"dataset runtime: {round((time.time() - starttime)/60, 2)} min")

# save datsets to file
utils.write_jsonzip(res, os.path.join("data", "results.json.gz"))

print(f"Total runtime: {round((time.time() - start_all)/3600, 2)} hrs")

# Plot model performance across all configurations

In [ ]:
# read datasets
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
# read model results
rdf = utils.get_all_model_results()


# get all model and input types
models = ["linear", "RF", "NN"]
input_types = ["single", "all", "original"]
split_types = ["LOCO", "Random"]
bar_colors = ["mediumorchid", "green", "darkorange", "gray"] * 3
ALPHA = 0.5
legend_colors = {"single": "mediumorchid", "all": "green", "original": "darkorange"}

fig, ax = plt.subplots(nrows=3, ncols=3)
ax = np.ravel(ax)
ii = 0
HEIGHT = 0.8

results_table = []

# loop over each dataset
for dsn in rdf["dataset"].unique():

    # df = pd.DataFrame(json.loads(ds[dsn]["df"]))
    t = ds[dsn]["target"]

    for split_type in ["LOCO", "Random"]:

        ndmes = []
        stds = []
        results_table_row = {}

        for model in models:
            for ip in input_types:

                # df00 = df0[(df0["model"] == m) & (df0["input_type"] == ip)]
                # err_df0.loc[ip, m] = float(df00["ndme_median"])
                # err_df0.loc[ip, m+'_std'] = float(df00["ndme_std"])
                rdf0 = rdf[
                    (rdf["dataset"] == dsn)
                    & (rdf["model"] == model)
                    & (rdf["input_type"] == ip)
                    & (rdf["split_type"] == split_type)
                ]

                split_ndmes = []

                for split in rdf0["split"].unique():
                    rdf00 = rdf0[rdf0["split"] == split]

                    rmse = np.sqrt(
                        np.mean(np.square(rdf00["predicted"] - rdf00["actual"]))
                    )
                    ndme = rmse / np.std(rdf00["actual"])

                    split_ndmes.append(ndme)

                ndme_median = np.median(split_ndmes)
                ndmes.append(ndme_median)
                stds.append(np.std(split_ndmes))
                results_table_row[f"{ip} - {model}"] = ndme_median

        if split_type == "LOCO":
            results_table.append(results_table_row)

        low_ndme = -np.min(ndmes) if split_type == "LOCO" else np.min(ndmes)
        ax[ii].axvline(x=low_ndme, c="purple", alpha=ALPHA, lw=0.5)

        ax[ii].axvline(x=-1, c="gray", alpha=ALPHA, lw=0.5, linestyle="dotted")
        ax[ii].axvline(x=1, c="gray", alpha=ALPHA, lw=0.5, linestyle="dotted")

        # insert 0's in dadta so there's gaps between split_type in the plot
        for data in [ndmes, stds]:
            data.insert(3, 0)
            data.insert(7, 0)

        bars = ax[ii].barh(
            np.arange(len(ndmes)),
            width=ndmes if split_type == "Random" else np.negative(ndmes),
            lw=0,
            height=HEIGHT,
            alpha=ALPHA,
            xerr=stds,
            error_kw=dict(
                lw=0.5,
                capsize=0,
                capthick=0,
                alpha=0.3,
            ),
        )

        for bi, b in enumerate(bars):
            b.set_color(bar_colors[bi])

    ax[ii].axvline(x=0, c="k", lw=0.5)
    ax[ii].set_title(t, fontsize=FONTSIZE)
    ax[ii].set_yticks([1, 5, 9])
    ax[ii].set_xlim([-1.5, 1.5])
    ax[ii].set_xticks([-1.5, -1, -0.5, 0, 0.5, 1, 1.5])
    if ii > 5:
        ax[ii].set_xlabel("NDME")
        # ax[ii].set_xticks([-1.5, -1, -0.5, 0, 0.5, 1, 1.5], [1.5, 1, 0.5, 0, 0.5, 1, 1.5])
        ax[ii].set_xticklabels(["1.5", "1", "0.5", "0", "0.5", "1", "1.5"])
    else:
        ax[ii].set_xticklabels([])

    if ii in [0, 3, 6]:
        ax[ii].set_yticklabels(models)
    else:
        ax[ii].set_yticklabels([])

    # horizontal split_type label
    ax[ii].text(
        0.02,
        0.96,
        "Extrap.\n(LOCO)",
        color="k",
        fontsize=FONTSIZE - 3,
        ha="left",
        va="top",
        transform=ax[ii].transAxes,
    )
    ax[ii].text(
        0.98,
        0.96,
        "Interp.\n(random)",
        color="k",
        fontsize=FONTSIZE - 3,
        ha="right",
        va="top",
        transform=ax[ii].transAxes,
    )

    ii += 1

ax[1].legend(
    ncol=3,
    bbox_to_anchor=(0.5, 1.25),
    loc="lower center",
    # fontsize=16,
    handles=[
        mpatches.Patch(
            color=legend_colors[k],
            alpha=ALPHA,
            label={"single": "BE", "all": "BE + original", "original": "original"}[k],
            lw=0,
        )
        for k in legend_colors
    ],
)
plt.subplots_adjust(wspace=-0.95, hspace=1)
plt.tight_layout()
plt.gcf().savefig(os.path.join(utils.FIG_BASEPATH, "ModelErrorSummary.png"))
plt.show()


results_table = pd.DataFrame(results_table)
results_table.index = [ds[dsn]["target"] for dsn in rdf["dataset"].unique()]
results_table = results_table.T
results_table

# Extrapolation and interpolation performance

In [ ]:
# read datasets
ds = utils.read_jsonzip(os.path.join("data", "datasets.json.gz"))
# read result sets
rs = pd.DataFrame(utils.read_jsonzip(os.path.join("data", "results.json.gz")))

# read model results
rdf = utils.get_all_model_results()


models = ["RF", "NN"]
input_types = ["original"]
ALPHA = 0.5

fig, ax = plt.subplots(nrows=1, ncols=2)
ax = np.ravel(ax)

colors = {
    "melting temp": "dodgerblue",
    "bulk modulus": "tomato",
    "band gap": "darkgreen",
    "heat capacity": "darkorange",
    "compressive strength": "navy",
    "formation energy": "magenta",
    "fish weight": "darkred",
    "airfoil sound": "limegreen",
    "abalone rings": "gold",
}

# colors = {"original": 'blue', 'single': 'green', 'all': 'red'}
# colors = {"NN": "dodgerblue", "RF": "tomato"}
markers = {"NN": "o", "RF": "s"}


for ii, split_type in enumerate(["Random", "LOCO"]):

    all_x, all_y = [], []

    # loop over each dataset
    for dsn in rs["dataset"].unique():
        df = pd.DataFrame(json.loads(ds[dsn]["df"]))
        t = ds[dsn]["target"]

        # get NDME for the single-feature linear models
        sflm_df = rdf[
            (rdf["dataset"] == dsn)
            & (rdf["model"] == "linear")
            & (rdf["input_type"] == "single")
            & (rdf["split_type"] == split_type)
        ]
        sflm_ndmes = []
        for split in sflm_df["split"].unique():
            sflm_rdf0 = sflm_df[sflm_df["split"] == split]
            rmse = np.sqrt(
                np.mean(np.square(sflm_rdf0["predicted"] - sflm_rdf0["actual"]))
            )
            ndme = rmse / np.std(sflm_rdf0["actual"])
            sflm_ndmes.append(ndme)

        for model in models:
            for ip in input_types:

                rdf0 = rdf[
                    (rdf["dataset"] == dsn)
                    & (rdf["model"] == model)
                    & (rdf["input_type"] == ip)
                    & (rdf["split_type"] == split_type)
                ]
                compare_ndmes = []
                for split in rdf0["split"].unique():
                    rdf00 = rdf0[rdf0["split"] == split]
                    rmse = np.sqrt(
                        np.mean(np.square(rdf00["predicted"] - rdf00["actual"]))
                    )
                    ndme = rmse / np.std(rdf00["actual"])
                    compare_ndmes.append(ndme)

                ax[ii].scatter(
                    compare_ndmes,
                    sflm_ndmes,
                    label=f"{model} - {ip}",
                    s=10,
                    c=colors[t],
                    marker=markers[model],
                    edgecolor="w",
                    lw=0.3,
                    alpha=ALPHA,
                )

                all_y += sflm_ndmes
                all_x += compare_ndmes

    # plot a confidence interval
    [
        utils.confidence_ellipse(
            all_x,
            all_y,
            ax[ii],
            n_std=contour[0],
            facecolor="none",
            lw=0.5,
            edgecolor="gray",
            linestyle=contour[1],
            zorder=0,
        )
        for contour in [
            (1, "solid"),  # (2, 'dashed'),# (3, 'dotted'),
        ]
    ]

    medx, medy = np.median(all_x), np.median(all_y)
    # medx, medy = np.std(all_x), np.std(all_y)

    lin_frac = sum([all_y[i] > all_x[i] for i in range(len(all_x))]) / len(all_x)
    print(split_type)
    print(f"Fraction in which blackbox achieved lower error: {lin_frac}")

    ax[ii].scatter(medx, medy, c="k", s=100, marker="+", lw=0.5)
    print(f"median point: {medx, medy}")
    print(f"percent difference: {(medy - medx)/medx}")

    ax[ii].plot([0, 1.5], [0, 1.5], lw=0.5, color="gray")

    label = (
        "Extrapolation (LOCO CV)"
        if split_type == "LOCO"
        else "Interpolation (random CV)"
    )
    ax[ii].set_xlim([0, 1.5])
    ax[ii].set_ylim([0, 1.5])
    ax[ii].set_xlabel("NDME (black-box model)", fontsize=FONTSIZE)
    ax[ii].set_ylabel("NDME (single-feat. lin. reg.)", fontsize=FONTSIZE)
    ax[ii].set_title(
        label,
        fontsize=FONTSIZE,
    )
    ax[ii].axhline(y=1, c="gray", alpha=ALPHA, lw=0.5, linestyle="dotted")
    ax[ii].axvline(x=1, c="gray", alpha=ALPHA, lw=0.5, linestyle="dotted")


ax[0].legend(
    ncol=1,
    bbox_to_anchor=(1, 0),
    loc="lower right",
    fontsize=FONTSIZE - 6,  # facecolor='white', framealpha=1,
    handles=[
        mpatches.Patch(color=colors[k], alpha=ALPHA, label=k, lw=0) for k in colors
    ],
)

fig.set_size_inches((5, 2.5))
plt.tight_layout()
plt.gcf().savefig(
    os.path.join(utils.FIG_BASEPATH, "ExtrapolationAndInterpolationNDME.png")
)
plt.show()